In [1]:
!pip install opencv-python numpy 

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\juanc\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np

def nothing(x):
    pass

# 1) Carga y gris
img_bgr = cv2.imread('Datos/Payday2.jpg')
img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

# 2) Ventana y trackbars
win = 'Control de filtros'
cv2.namedWindow(win)

cv2.createTrackbar('Filtro', win, 0, 9, nothing)
cv2.createTrackbar('Kernel', win, 1, 31, nothing)

while True:
    key = cv2.waitKey(100) & 0xFF
    if key == 27:  # ESC
        break

    mode   = cv2.getTrackbarPos('Filtro', win)
    ksize  = cv2.getTrackbarPos('Kernel', win)

    # Asegurar kernel impar y ≥1
    if ksize < 1: ksize = 1
    if ksize % 2 == 0: ksize += 1

    # Selección de filtro
    if mode == 0:
        # Blur manual
        kernel = np.ones((ksize, ksize), np.float32) / (ksize**2)
        out = cv2.filter2D(img_gray, -1, kernel)

    elif mode == 1:
        # Gaussian Blur
        out = cv2.GaussianBlur(img_gray, (ksize, ksize), sigmaX=0)

    elif mode == 2:
        # Median Blur
        out = cv2.medianBlur(img_gray, ksize)

    elif mode == 3:
        # Bilateral Filter
        # Diametro = ksize, sigmaColor y sigmaSpace = ksize*2
        out = cv2.bilateralFilter(img_gray, ksize, ksize*2, ksize*2)

    elif mode == 4:
        # Sharpening manual (fijo 3×3)
        kernel = np.array([[ 0, -1,  0],
                           [-1,  5, -1],
                           [ 0, -1,  0]], np.float32)
        out = cv2.filter2D(img_gray, -1, kernel)

    elif mode == 5:
        # Unsharp Mask
        blur = cv2.GaussianBlur(img_gray, (ksize, ksize), sigmaX=0)
        out = cv2.addWeighted(img_gray, 1.5, blur, -0.5, 0)

    elif mode == 6:
        # Sobel X
        sobel = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=ksize)
        out = cv2.convertScaleAbs(sobel)

    elif mode == 7:
        # Sobel Y
        sobel = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=ksize)
        out = cv2.convertScaleAbs(sobel)

    elif mode == 8:
        # Sobel combinado
        sx = cv2.Sobel(img_gray, cv2.CV_64F, 1, 0, ksize=ksize)
        sy = cv2.Sobel(img_gray, cv2.CV_64F, 0, 1, ksize=ksize)
        out = cv2.bitwise_or(cv2.convertScaleAbs(sx),
                             cv2.convertScaleAbs(sy))

    else:
        # Laplaciano
        lap = cv2.Laplacian(img_gray, cv2.CV_64F, ksize=ksize)
        out = cv2.convertScaleAbs(lap)



    # Mostrar lado a lado
    combo = np.hstack((img_gray, out))
    cv2.imshow(win, combo)

cv2.destroyAllWindows()



error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2564: error: (-27:Null pointer) NULL window: 'Control de filtros' in function 'cvGetTrackbarPos'
